In [16]:
class CacheBlock:
    def __init__(self, tag, data):
        self.tag = tag
        self.data = data
        self.valid = True

class NWaySetAssociativeCache:
    def __init__(self, cache_size, block_size, ways):
        self.cache_size = cache_size
        self.block_size = block_size
        self.ways = ways
        self.sets = cache_size // (block_size * ways)
        # print(self.sets)
        self.cache = [[None for _ in range(ways)] for _ in range(self.sets)]
        # print(self.cache, len(self.cache))

    def _index_and_tag(self, address):
        block_number = address // self.block_size
        index = block_number % self.sets
        tag = block_number // self.sets
        # print(block_number, self.sets)
        # print(block_number, index, tag)
        return index, tag

    def access(self, address, data=None):
        index, tag = self._index_and_tag(address)
        set_ = self.cache[index]
        print(index, tag, set_)

        print("retrieving")
        # Search for the block in the set
        for block in set_:
            if block and block.tag == tag:
                if data is not None:
                    block.data = data
                return block.data  # Cache hit

        print("setting")
        # Cache miss: find an empty slot or replace using a policy (e.g., LRU)
        for i in range(self.ways):
            if not set_[i]:
                set_[i] = CacheBlock(tag, data)
                return None  # Cache miss

        print("replace")
        # Replace the first block (simple replacement policy)
        set_[0] = CacheBlock(tag, data)
        return None  # Cache miss

In [18]:
# Example usage:
cache = NWaySetAssociativeCache(cache_size=1024, block_size=64, ways=4)
print(cache.access(100))  # Miss
print("again!")
cache.access(100, data='A')  # Insert
print(cache.access(100))  # Hit, returns 'A'

1 0 [None, None, None, None]
retrieving
setting
None
again!
1 0 [<__main__.CacheBlock object at 0x107b1b440>, None, None, None]
retrieving
1 0 [<__main__.CacheBlock object at 0x107b1b440>, None, None, None]
retrieving
A


In [43]:
from collections import deque

class NWaySetAssociativeCacheLRU:
    def __init__(self, cache_size, block_size, ways):
        self.cache_size = cache_size
        self.block_size = block_size
        self.ways = ways
        self.sets = cache_size // (block_size * ways)
        print(self.sets)
        self.cache = [[None for _ in range(ways)] for _ in range(self.sets)]
        self.lru = [deque(maxlen=ways) for _ in range(self.sets)]
        print(self.cache)

    def _index_and_tag(self, address):
        block_number = address // self.block_size
        print(block_number) # 3  
        index = block_number % self.sets ## which n-way you are at  ## 3 
        tag = block_number // self.sets ## which set out of sets of cache  ## 0 
        return index, tag

    def access(self, address, data=None):
        index, tag = self._index_and_tag(address) ## 1, 0
        set_ = self.cache[index]
        print(index, tag, set_)
        # print
        lru_queue = self.lru[index]

        # Search for the block in the set
        for i, block in enumerate(set_):
            if block and block.tag == tag:
                if data is not None:
                    block.data = data
                lru_queue.remove(i) ## [1, 2, 3, 4] 
                lru_queue.append(i)
                return block.data  # Cache hit

        # Cache miss: find an empty slot or replace using LRU
        for i in range(self.ways):
            if not set_[i]:
                set_[i] = CacheBlock(tag, data)
                lru_queue.append(i)
                return None  # Cache miss

        # Replace the LRU block
        lru_index = lru_queue.popleft()
        set_[lru_index] = CacheBlock(tag, data)
        lru_queue.append(lru_index)
        return None  # Cache miss

In [46]:
# Example usage:
cache = NWaySetAssociativeCacheLRU(cache_size=1024, block_size=64, ways=2) ## 8 sets of 2-way cache 
print(cache.access(64 * 3 + 1))  # Miss
# cache.access(100, data='A')  # Insert
# print(cache.access(100))  # Hit, returns 'A'


8
[[None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None]]
3
3 0 [None, None]
None
